In [58]:
import pandas as pd
import numpy as np
shots1 = pd.read_csv('game_shots_20116.csv')
shots2 = pd.read_csv('game_shots_21006.csv')


shots1['Shooter'] = shots1['Shooter'].astype('float')
shots1['A1'] = shots1['A1'].astype('float')
shots1['A2'] = shots1['A2'].astype('float')
shots1['A3'] = shots1['A3'].astype('float')

shots2['Shooter'] = shots2['Shooter'].astype('float')
shots2['A1'] = shots2['A1'].astype('float')
shots2['A2'] = shots2['A2'].astype('float')
shots2['A3'] = shots2['A3'].astype('float')



shots = pd.concat([shots1, shots2],ignore_index=True)

lookup = pd.read_csv('allPlayersLookup.csv')
lookup = lookup.dropna(subset=['primaryNumber'])

# bare minimum needed to find player id
sub_lookup = lookup[['playerId', 'team', 'primaryNumber']]


players = pd.read_csv('skaters_2022.csv')
players = players.loc[players['situation'] == 'all']

# Set display options to show all columns and a large width
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Adjust the width as needed

In [8]:
for team in list(shots['Team'].unique()):
 assert team in list(lookup['team'].unique())

In [9]:
def append_player_id(shots_df, lookup_df, col_name):
    """
    Append ShooterPlayerId column to shots dataframe by looking up (Team, Shooter) in lookup dataframe.
    
    Parameters:
    shots_df (pd.DataFrame): DataFrame containing shot data with 'Team' and 'Shooter' columns
    lookup_df (pd.DataFrame): DataFrame containing player lookup data with 'playerId', 'team', 'primaryNumber' columns
    
    Returns:
    tuple: (DataFrame with new ShooterPlayerId column, list of failed lookups)
    """
    # Create copy to avoid modifying original
    result_df = shots_df.copy()
    
    # Create dictionary for faster lookups
    player_dict = {}
    failed_lookups = []
    
    # Convert lookup dataframe to dictionary with (team, number) as key
    for _, row in lookup_df.iterrows():
        player_dict[(row['team'], row['primaryNumber'])] = row['playerId']
    
    # Function to lookup player ID
    def get_player_id(row):
        lookup_key = (row['Team'], row[col_name])
        player_id = player_dict.get(lookup_key)
        
        if player_id is None:
            failed_lookups.append({
                'Team': row['Team'],
                col_name: row[col_name],
                'Game ID': row['Game ID'],
                'Period': row['Period'],
                'Time': row['Time']
            })
            return None
        return player_id
    
    # Apply lookup to each row
    naming = col_name+'PlayerId'
    result_df[naming] = result_df.apply(get_player_id, axis=1)
    
    # Convert failed lookups to DataFrame for easier analysis
    failed_lookups_df = pd.DataFrame(failed_lookups)
    
    return result_df, failed_lookups_df

use function and determine which composite keys need manual filling in

In [59]:
result_s, failed = append_player_id(shots, sub_lookup, 'Shooter')


failed['composite_key'] = failed['Team'].astype(str) + '_' + failed['Shooter'].astype(str)
print( f"Number of unique failueres: {failed['composite_key'].nunique()}")
unique1 = list(failed['composite_key'].unique())


result_a1, failed = append_player_id(shots, sub_lookup, 'A1')


failed['composite_key'] = failed['Team'].astype(str) + '_' + failed['A1'].astype(str)
print( f"Number of unique failueres: {failed['composite_key'].nunique()}")
unique2 = list(failed['composite_key'].unique())


result_a2, failed = append_player_id(shots, sub_lookup, 'A2')


failed['composite_key'] = failed['Team'].astype(str) + '_' + failed['A2'].astype(str)
print( f"Number of unique failueres: {failed['composite_key'].nunique()}")
unique3 = list(failed['composite_key'].unique())



result_a3, failed = append_player_id(shots, sub_lookup, 'A3')
failed['composite_key'] = failed['Team'].astype(str) + '_' + failed['A3'].astype(str)
print( f"Number of unique failueres: {failed['composite_key'].nunique()}")
unique4 = list(failed['composite_key'].unique())

uniques = list(set(unique1 + unique2 + unique3 + unique4))
uniques = [i for i in uniques if i.endswith('_nan') is False]
uniques.sort()
uniques

Number of unique failueres: 8
Number of unique failueres: 12
Number of unique failueres: 9
Number of unique failueres: 7


['BUF_12.0',
 'BUF_19.0',
 'BUF_53.0',
 'CHI_4.0',
 'CHI_82.0',
 'CHI_88.0',
 'CHI_89.0',
 'EDM_18.0',
 'EDM_71.0']

In [46]:
teams = ['BUF','BUF','BUF','CHI','CHI','CHI','CHI','EDM','EDM']
numbers = [12, 19, 53, 4, 82, 88, 89, 18, 71]
names = [
    'Jordan Greenway',
    'Peyton Krebs',
    'Jeff Skinner',
    'Seth Jones',
    'Caleb Jones',
    'Patrick Kane',
    'Andreas Athanasiou',
    'Zach Hyman',
    'Ryan McLeod'
]

m_fills = pd.DataFrame({'team':teams,'primaryNumber':numbers,'name':names})
m_fills['primaryNumber'] = m_fills['primaryNumber'].astype('float') 

In [51]:
mapping = pd.merge(left=m_fills, right=players, how='inner', on='name')
mapping.head(20)

,team_x,primaryNumber,name,playerId,season,team_y,position,situation,games_played,icetime,shifts,gameScore,onIce_xGoalsPercentage,offIce_xGoalsPercentage,onIce_corsiPercentage,offIce_corsiPercentage,onIce_fenwickPercentage,offIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_xFreeze,I_F_xPlayStopped,I_F_xPlayContinuedInZone,I_F_xPlayContinuedOutsideZone,I_F_flurryAdjustedxGoals,I_F_scoreVenueAdjustedxGoals,I_F_flurryScoreVenueAdjustedxGoals,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotsOnGoal,I_F_missedShots,I_F_blockedShotAttempts,I_F_shotAttempts,I_F_points,I_F_goals,I_F_rebounds,I_F_reboundGoals,I_F_freeze,I_F_playStopped,I_F_playContinuedInZone,I_F_playContinuedOutsideZone,I_F_savedShotsOnGoal,I_F_savedUnblockedShotAttempts,penalties,I_F_penalityMinutes,I_F_faceOffsWon,I_F_hits,I_F_takeaways,I_F_giveaways,I_F_lowDangerShots,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_scoreAdjustedShotsAttempts,I_F_unblockedShotAttempts,I_F_scoreAdjustedUnblockedShotAttempts,I_F_dZoneGiveaways,I_F_xGoalsFromxReboundsOfShots,I_F_xGoalsFromActualReboundsOfShots,I_F_reboundxGoals,I_F_xGoals_with_earned_rebounds,I_F_xGoals_with_earned_rebounds_scoreAdjusted,I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,I_F_shifts,I_F_oZoneShiftStarts,I_F_dZoneShiftStarts,I_F_neutralZoneShiftStarts,I_F_flyShiftStarts,I_F_oZoneShiftEnds,I_F_dZoneShiftEnds,I_F_neutralZoneShiftEnds,I_F_flyShiftEnds,faceoffsWon,faceoffsLost,timeOnBench,penalityMinutes,penalityMinutesDrawn,penaltiesDrawn,shotsBlockedByPlayer,OnIce_F_xOnGoal,OnIce_F_xGoals,OnIce_F_flurryAdjustedxGoals,OnIce_F_scoreVenueAdjustedxGoals,OnIce_F_flurryScoreVenueAdjustedxGoals,OnIce_F_shotsOnGoal,OnIce_F_missedShots,OnIce_F_blockedShotAttempts,OnIce_F_shotAttempts,OnIce_F_goals,OnIce_F_rebounds,OnIce_F_reboundGoals,OnIce_F_lowDangerShots,OnIce_F_mediumDangerShots,OnIce_F_highDangerShots,OnIce_F_lowDangerxGoals,OnIce_F_mediumDangerxGoals,OnIce_F_highDangerxGoals,OnIce_F_lowDangerGoals,OnIce_F_mediumDangerGoals,OnIce_F_highDangerGoals,OnIce_F_scoreAdjustedShotsAttempts,OnIce_F_unblockedShotAttempts,OnIce_F_scoreAdjustedUnblockedShotAttempts,OnIce_F_xGoalsFromxReboundsOfShots,OnIce_F_xGoalsFromActualReboundsOfShots,OnIce_F_reboundxGoals,OnIce_F_xGoals_with_earned_rebounds,OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OnIce_A_xOnGoal,OnIce_A_xGoals,OnIce_A_flurryAdjustedxGoals,OnIce_A_scoreVenueAdjustedxGoals,OnIce_A_flurryScoreVenueAdjustedxGoals,OnIce_A_shotsOnGoal,OnIce_A_missedShots,OnIce_A_blockedShotAttempts,OnIce_A_shotAttempts,OnIce_A_goals,OnIce_A_rebounds,OnIce_A_reboundGoals,OnIce_A_lowDangerShots,OnIce_A_mediumDangerShots,OnIce_A_highDangerShots,OnIce_A_lowDangerxGoals,OnIce_A_mediumDangerxGoals,OnIce_A_highDangerxGoals,OnIce_A_lowDangerGoals,OnIce_A_mediumDangerGoals,OnIce_A_highDangerGoals,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts
0,BUF,12.0,Jordan Greenway,8478413,2022,BUF,L,all,62,50238.0,1068.0,9.53,0.48,0.51,0.46,0.52,0.47,0.52,488.0,107.13,12.17,7.49,24.10,3.34,54.94,39.96,11.64,12.24,11.70,2.0,3.0,99.0,43.0,48.0,190.0,11.0,6.0,13.0,2.0,26.0,2.0,50.0,45.0,93.0,136.0,14.0,26.0,2.0,107.0,12.0,13.0,94.0,35.0,13.0,3.11,4.55,4.51,1.0,2.0,3.0,190.68,142.0,142.74,4.0,1.62,1.60,2.77,11.03,11.11,10.89,1068.0,137.0,149.0,176.0,606.0,188.0,161.0,134.0,585.0,2.0,12.0,177508.0,2

In [68]:
backup_lookup = mapping[['playerId', 'team_y', 'primaryNumber']].rename(columns={'team_y':'team'})

In [ ]:
sub_lookup_merged = pd.concat([sub_lookup, backup_lookup], ignore_index=True)
manual_row = pd.DataFrame({
    'playerId':[8474141],
    'team':['CHI'],
    'primaryNumber':[88.0]
})
sub_lookup_merged = pd.concat([sub_lookup_merged,manual_row], ignore_index=True)

In [74]:
chi = sub_lookup_merged.loc[sub_lookup_merged['team'] == 'CHI']
chi.loc[chi['primaryNumber'] == 88]

,playerId,team,primaryNumber
3139,8474141,CHI,88.0


In [76]:
sub_lookup

,playerId,team,primaryNumber
0,8478421,FLA,24.0
1,8477180,SJS,30.0
2,8465992,DET,44.0
3,8477932,FLA,5.0
4,8471451,WPG,21.0
...,...,...,...
3224,8475876,BOS,31.0
3225,8469760,ARI,4.0
3226,8465009,NYI,33.0
3227,8476878,TBL,28.0


In [137]:



result_s, failed_s = append_player_id(shots, sub_lookup_merged, 'Shooter')


# failed['composite_key'] = failed['team'].astype(str) + '_' + failed['Shooter'].astype(str)
# print( f"Number of unique failueres: {failed['composite_key'].nunique()}")
# unique1 = list(failed['composite_key'].unique())


result_a1, failed_a1 = append_player_id(shots, sub_lookup_merged, 'A1')


# failed['composite_key'] = failed['Team'].astype(str) + '_' + failed['A1'].astype(str)
# print( f"Number of unique failueres: {failed['composite_key'].nunique()}")
# unique2 = list(failed['composite_key'].unique())


result_a2, failed_a2 = append_player_id(shots, sub_lookup_merged, 'A2')


# failed['composite_key'] = failed['Team'].astype(str) + '_' + failed['A2'].astype(str)
# print( f"Number of unique failueres: {failed['composite_key'].nunique()}")
# unique3 = list(failed['composite_key'].unique())



result_a3, failed_a3 = append_player_id(shots, sub_lookup_merged, 'A3')
# failed['composite_key'] = failed['Team'].astype(str) + '_' + failed['A3'].astype(str)
# print( f"Number of unique failueres: {failed['composite_key'].nunique()}")
# unique4 = list(failed['composite_key'].unique())

# uniques = list(set(unique1 + unique2 + unique3 + unique4))
# uniques = [i for i in uniques if i.endswith('_nan') is False]
# uniques.sort()
# uniques

assert len(failed_a1) == shots['A1'].isna().sum()
assert len(failed_a2) == shots['A2'].isna().sum()
assert len(failed_a3) == shots['A3'].isna().sum()

assert result_s.shape == result_a1.shape == result_a2.shape == result_a3.shape

In [138]:
result_s['A1PlayerId'] = result_a1['A1PlayerId']
result_s['A2PlayerId'] = result_a2['A2PlayerId']
result_s['A3PlayerId'] = result_a3['A3PlayerId']

In [139]:
result_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Period            206 non-null    int64  
 1   Time              206 non-null    object 
 2   Strength          206 non-null    object 
 3   Team              206 non-null    object 
 4   Shooter           206 non-null    float64
 5   Shot Type?        206 non-null    object 
 6   A1                173 non-null    float64
 7   A2                92 non-null     float64
 8   A3                37 non-null     float64
 9   A1 Zone           173 non-null    object 
 10  A2 Zone           92 non-null     object 
 11  A3 Zone           37 non-null     object 
 12  SC?               81 non-null     object 
 13  SOG?              134 non-null    object 
 14  Screen            11 non-null     object 
 15  Situation         148 non-null    object 
 16  Origin            144 non-null    object 
 1

In [140]:
result_s.to_csv('merged_shots.csv', index=False)

In [135]:
player_stats = players[['playerId', 'OnIce_F_xGoals']]
m1 = (
    pd
    .merge(left=result_s, right=player_stats, how='left', left_on='ShooterPlayerId', right_on='playerId')
    .rename(columns={'OnIce_F_xGoals':'ShooterxGoals'})
    .drop(columns=['playerId'])
)
m2 = (
    pd
    .merge(left=m1, right=player_stats, how='left', left_on='A1PlayerId', right_on='playerId')
    .rename(columns={'OnIce_F_xGoals':'A1xGoals'})
    .drop(columns=['playerId'])
)
m3 = (
    pd
    .merge(left=m2, right=player_stats, how='left', left_on='A2PlayerId', right_on='playerId')
    .rename(columns={'OnIce_F_xGoals':'A2xGoals'})
    .drop(columns=['playerId'])
)
m4 = (
    pd
    .merge(left=m3, right=player_stats, how='left', left_on='A3PlayerId', right_on='playerId')
    .rename(columns={'OnIce_F_xGoals':'A3xGoals'})
    .drop(columns=['playerId'])
)


In [136]:
m4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Period            206 non-null    int64  
 1   Time              206 non-null    object 
 2   Strength          206 non-null    object 
 3   Team              206 non-null    object 
 4   Shooter           206 non-null    float64
 5   Shot Type?        206 non-null    object 
 6   A1                173 non-null    float64
 7   A2                92 non-null     float64
 8   A3                37 non-null     float64
 9   A1 Zone           173 non-null    object 
 10  A2 Zone           92 non-null     object 
 11  A3 Zone           37 non-null     object 
 12  SC?               81 non-null     object 
 13  SOG?              134 non-null    object 
 14  Screen            11 non-null     object 
 15  Situation         148 non-null    object 
 16  Origin            144 non-null    object 
 1

In [102]:
result_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Period            206 non-null    int64  
 1   Time              206 non-null    object 
 2   Strength          206 non-null    object 
 3   Team              206 non-null    object 
 4   Shooter           206 non-null    float64
 5   Shot Type?        206 non-null    object 
 6   A1                173 non-null    float64
 7   A2                92 non-null     float64
 8   A3                37 non-null     float64
 9   A1 Zone           173 non-null    object 
 10  A2 Zone           92 non-null     object 
 11  A3 Zone           37 non-null     object 
 12  SC?               81 non-null     object 
 13  SOG?              134 non-null    object 
 14  Screen            11 non-null     object 
 15  Situation         148 non-null    object 
 16  Origin            144 non-null    object 
 1

In [104]:
players.head()

,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,onIce_xGoalsPercentage,offIce_xGoalsPercentage,onIce_corsiPercentage,offIce_corsiPercentage,onIce_fenwickPercentage,offIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_xFreeze,I_F_xPlayStopped,I_F_xPlayContinuedInZone,I_F_xPlayContinuedOutsideZone,I_F_flurryAdjustedxGoals,I_F_scoreVenueAdjustedxGoals,I_F_flurryScoreVenueAdjustedxGoals,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotsOnGoal,I_F_missedShots,I_F_blockedShotAttempts,I_F_shotAttempts,I_F_points,I_F_goals,I_F_rebounds,I_F_reboundGoals,I_F_freeze,I_F_playStopped,I_F_playContinuedInZone,I_F_playContinuedOutsideZone,I_F_savedShotsOnGoal,I_F_savedUnblockedShotAttempts,penalties,I_F_penalityMinutes,I_F_faceOffsWon,I_F_hits,I_F_takeaways,I_F_giveaways,I_F_lowDangerShots,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_scoreAdjustedShotsAttempts,I_F_unblockedShotAttempts,I_F_scoreAdjustedUnblockedShotAttempts,I_F_dZoneGiveaways,I_F_xGoalsFromxReboundsOfShots,I_F_xGoalsFromActualReboundsOfShots,I_F_reboundxGoals,I_F_xGoals_with_earned_rebounds,I_F_xGoals_with_earned_rebounds_scoreAdjusted,I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,I_F_shifts,I_F_oZoneShiftStarts,I_F_dZoneShiftStarts,I_F_neutralZoneShiftStarts,I_F_flyShiftStarts,I_F_oZoneShiftEnds,I_F_dZoneShiftEnds,I_F_neutralZoneShiftEnds,I_F_flyShiftEnds,faceoffsWon,faceoffsLost,timeOnBench,penalityMinutes,penalityMinutesDrawn,penaltiesDrawn,shotsBlockedByPlayer,OnIce_F_xOnGoal,OnIce_F_xGoals,OnIce_F_flurryAdjustedxGoals,OnIce_F_scoreVenueAdjustedxGoals,OnIce_F_flurryScoreVenueAdjustedxGoals,OnIce_F_shotsOnGoal,OnIce_F_missedShots,OnIce_F_blockedShotAttempts,OnIce_F_shotAttempts,OnIce_F_goals,OnIce_F_rebounds,OnIce_F_reboundGoals,OnIce_F_lowDangerShots,OnIce_F_mediumDangerShots,OnIce_F_highDangerShots,OnIce_F_lowDangerxGoals,OnIce_F_mediumDangerxGoals,OnIce_F_highDangerxGoals,OnIce_F_lowDangerGoals,OnIce_F_mediumDangerGoals,OnIce_F_highDangerGoals,OnIce_F_scoreAdjustedShotsAttempts,OnIce_F_unblockedShotAttempts,OnIce_F_scoreAdjustedUnblockedShotAttempts,OnIce_F_xGoalsFromxReboundsOfShots,OnIce_F_xGoalsFromActualReboundsOfShots,OnIce_F_reboundxGoals,OnIce_F_xGoals_with_earned_rebounds,OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OnIce_A_xOnGoal,OnIce_A_xGoals,OnIce_A_flurryAdjustedxGoals,OnIce_A_scoreVenueAdjustedxGoals,OnIce_A_flurryScoreVenueAdjustedxGoals,OnIce_A_shotsOnGoal,OnIce_A_missedShots,OnIce_A_blockedShotAttempts,OnIce_A_shotAttempts,OnIce_A_goals,OnIce_A_rebounds,OnIce_A_reboundGoals,OnIce_A_lowDangerShots,OnIce_A_mediumDangerShots,OnIce_A_highDangerShots,OnIce_A_lowDangerxGoals,OnIce_A_mediumDangerxGoals,OnIce_A_highDangerxGoals,OnIce_A_lowDangerGoals,OnIce_A_mediumDangerGoals,OnIce_A_highDangerGoals,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts
1,8471817,2022,Ryan Reaves,MIN,R,all,73,40825.0,889.0,8.58,0.45,0.52,0.45,0.52,0.46,0.53,814.0,49.47,6.70,3.80,11.20,1.72,26.57,19.02,6.55,6.77,6.62,5.0,5.0,48.0,21.0,10.0,79.0,15.0,5.0,5.0,1.0,7.0,1.0,24.0,27.0,43.0,64.0,11.0,43.0,4.0,196.0,15.0,16.0,33.0,29.0,7.0,1.16,3.58,1.96,0.0,4.0,1.0,80.30,69.0,69.70,10.0,0.80,1.15,1.90,5.60,5.68,5.59,889.0,70.0,91.0,141.0,587.0,138.0,140.0,98.0,513.0,4.0,11.0,227036.0,43.0,47.0,13.0,24.0,296.11,23.65,23.03,24.21,23.56,284.0,

In [ ]:
result_s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Period            19 non-null     int64  
 1   Time              19 non-null     object 
 2   Strength          19 non-null     object 
 3   Team              19 non-null     object 
 4   Shooter           19 non-null     float64
 5   Shot Type?        19 non-null     object 
 6   A1                19 non-null     float64
 7   A2                19 non-null     float64
 8   A3                8 non-null      float64
 9   A1 Zone           19 non-null     object 
 10  A2 Zone           19 non-null     object 
 11  A3 Zone           8 non-null      object 
 12  SC?               7 non-null      object 
 13  SOG?              13 non-null     object 
 14  Screen            4 non-null      object 
 15  Situation         6 non-null      object 
 16  Origin            6 non-null      object 
 17 

In [106]:
list(players.columns)

['playerId',
 'season',
 'name',
 'team',
 'position',
 'situation',
 'games_played',
 'icetime',
 'shifts',
 'gameScore',
 'onIce_xGoalsPercentage',
 'offIce_xGoalsPercentage',
 'onIce_corsiPercentage',
 'offIce_corsiPercentage',
 'onIce_fenwickPercentage',
 'offIce_fenwickPercentage',
 'iceTimeRank',
 'I_F_xOnGoal',
 'I_F_xGoals',
 'I_F_xRebounds',
 'I_F_xFreeze',
 'I_F_xPlayStopped',
 'I_F_xPlayContinuedInZone',
 'I_F_xPlayContinuedOutsideZone',
 'I_F_flurryAdjustedxGoals',
 'I_F_scoreVenueAdjustedxGoals',
 'I_F_flurryScoreVenueAdjustedxGoals',
 'I_F_primaryAssists',
 'I_F_secondaryAssists',
 'I_F_shotsOnGoal',
 'I_F_missedShots',
 'I_F_blockedShotAttempts',
 'I_F_shotAttempts',
 'I_F_points',
 'I_F_goals',
 'I_F_rebounds',
 'I_F_reboundGoals',
 'I_F_freeze',
 'I_F_playStopped',
 'I_F_playContinuedInZone',
 'I_F_playContinuedOutsideZone',
 'I_F_savedShotsOnGoal',
 'I_F_savedUnblockedShotAttempts',
 'penalties',
 'I_F_penalityMinutes',
 'I_F_faceOffsWon',
 'I_F_hits',
 'I_F_takeaways